In [45]:
import re

EXAM_DOSE_SETTINGS = r'^Exam Dose Settings,$'
EMPTY_STRING = r'^$'

print(bool(re.match(EXAM_DOSE_SETTINGS, 'Exam Dose Settings,')))
print(bool(re.match(EMPTY_STRING, '')))
print(bool(re.match(EMPTY_STRING or EXAM_DOSE_SETTINGS, 'ADULT HEAD 1.1 Head (adult) 1.25')))

True
True
False


In [4]:
# Reading Protocol Names

PROTOCOL_NAME = r"^(?!Series)[^,^\n]+$"
print(bool(re.match(PROTOCOL_NAME, 'ADULT CHEST 5.7 Thorax_Contrast (adult) N(Free)+N(DIBH)+C(A+V) 1.25mm')))
print(not bool(re.match(PROTOCOL_NAME, 'ADULT CHEST 5.7 Thorax_Contrast (adult) N(Free)+N(DIBH)+C(A+V) 1.25mm,')))
print(not bool(re.match(PROTOCOL_NAME, 'Series 2 Group 1 Scan Settings\n')))
print(not bool(re.match(PROTOCOL_NAME, '\n')))


True
True
True
True


In [39]:
import unittest
import re

# from ge_ct_protocol_name import PROTOCOL_NAME_ADULT_HEAD

PROTOCOL_NAME_ADULT_HEAD = r"^(?=ADULT HEAD)\s"

# str1 = 'ADULT HEAD 1.1 Head (adult) 1.25'
str1 = 'ADULT HEAD '
bool(bool(re.match(PROTOCOL_NAME_ADULT_HEAD, str1)))

False

In [47]:
SERIES_NAME = r"^(?=Series)[^,^\n]+$"
print(not bool(re.match(SERIES_NAME,
      'ADULT CHEST 5.7 ThoraxE_Contrast (adult) N(Free)+N(DIBH)+C(A+V) 1.25mm')))
print(not bool(re.match(SERIES_NAME,
      'ADULT CHEST 5.7 Thorax_Contrast (adult) N(Free)+N(DIBH)+C(A+V) 1.25mm,')))
print(bool(re.match(SERIES_NAME, 'Series 2 Group 1 Scan Settings\n')))
print(bool(re.match(SERIES_NAME, 'Series 2 Group 1 Recon 1 Settings\n')))
print(not bool(re.match(SERIES_NAME, '\n')))
print(not bool(re.match(SERIES_NAME, 'Series 2,Axial,HeadFirst,Supine\n')))


True
True
True
True
True
True


In [48]:
def read_params_in_line(line):
    if line[-1] == ',':
        res = line[:-1].split(sep=',')
    else:
        res = line.split(sep=',')
    
    return res


line1 = 'ExamCtdi,ExamDLP,'
line2 = 'No,No,No,No'
line3 = 'AutoStore,Gating,SeriesLevelCopy,Injector,'
lines = [line1, line2, line3]

for line in lines:
    print(read_params_in_line(line))


['ExamCtdi', 'ExamDLP']
['No', 'No', 'No', 'No']
['AutoStore', 'Gating', 'SeriesLevelCopy', 'Injector']


In [49]:
import re 
EXAM_DOSE_SETTINGS = r'^Exam Dose Settings,$'
EMPTY_STRING = r'^[^\n]+$'
PROTOCOL_NAME = r"^(?!Series)[^,^\n]+$"


In [60]:
# init path variables
from pathlib import Path

INPUT = Path('./input')
OUTPUT = Path('./output')
PROTOCOLS_LIST = Path('./protocols_lists')

In [51]:
test_protocols = INPUT / 'Protocols reatding test.csv'

with open(test_protocols, 'r', encoding='utf8') as file:
    lines = file.readlines()

protocol_names_list = []

for line in lines[1:]: # the first symbol
    if bool(re.match(PROTOCOL_NAME, line)):
        protocol_names_list.append(line)

In [95]:
import re

EXAM_DOSE_SETTINGS = 'Exam Dose Settings'
SCAN_SETTINGS = 'Scan Settings'

SERIES_AXIAL = r'^Series \d+,Axial'
SERIES_SCAN_SETTINGS = r'^Series \d+ Group \d+ Scan Settings'
SERIES_RECON_SETTINGS = r'^Series \d+ Group \d+ Recon \d Settings'

SERIES = 'series'
ID = 'id'
TYPE = 'type'
RECON = 'recons'


protocols_path = INPUT / '2022-04-18- GE. Discovery.EXAM_ALL.CSV'

# filename = 'Protocols reatding test.csv'

with open(protocols_path, 'r') as file:
    lines = file.readlines()


def read_exams(fpath):
    with open(protocols_path, 'r') as file:
        lines = file.readlines()

    exams = {}

    for i in range(0, len(lines)):

        line = lines[i]

        if bool(re.match(PROTOCOL_NAME, line)):

            name = line
            name = name.strip('\n')
            exams[name] = {}
            exams[name][SERIES] = []
        else:
            if EXAM_DOSE_SETTINGS in line:
                i += 1
                line = lines[i]
                param = read_params_in_line(line)

                i += 1
                line = lines[i]
                vals = read_params_in_line(line)

                exams[name][EXAM_DOSE_SETTINGS] = dict(zip(param, vals))
                i += 1

            if bool(re.match(SERIES_AXIAL, line)):
                line = lines[i]
                param_series = read_params_in_line(line)

                if param_series[1] == 'Axial':
                    series_item = {}
                    series_item[ID] = param_series[0]
                    series_item[TYPE] = param_series[1]
                    series_item[RECON] = []
                    exams[name][SERIES].append(series_item)

                    i += 1

            if bool(re.match(SERIES_SCAN_SETTINGS, line)):
                i += 1
                line = lines[i]
                param = read_params_in_line(line)

                i += 1
                line = lines[i]
                vals = read_params_in_line(line)

                series_item[SCAN_SETTINGS] = dict(zip(param, vals))

            if bool(re.match(SERIES_RECON_SETTINGS, line)):
                i += 1
                line = lines[i]
                param = read_params_in_line(line)

                i += 1
                line = lines[i]
                vals = read_params_in_line(line)

                series_item[RECON].append(dict(zip(param, vals)))

    return exams


exams = read_exams(protocols_path)

# for i in range(0, len(lines)):

#     line = lines[i]

#     if bool(re.match(PROTOCOL_NAME, line)):

#         name = line
#         name = name.strip('\n')
#         exams[name] = {}
#         exams[name][SERIES] = []
#     else:
#         if EXAM_DOSE_SETTINGS in line:
#             i += 1
#             line = lines[i]
#             param = read_params_in_line(line)

#             i += 1
#             line = lines[i]
#             vals = read_params_in_line(line)

#             exams[name][EXAM_DOSE_SETTINGS] = dict(zip(param, vals))
#             i += 1

#         if bool(re.match(SERIES_AXIAL, line)):
#             line = lines[i]
#             param_series = read_params_in_line(line)

#             if param_series[1] == 'Axial':
#                 series_item = {}
#                 series_item[ID] = param_series[0]
#                 series_item[TYPE] = param_series[1]
#                 series_item[RECON] = []
#                 exams[name][SERIES].append(series_item)

#                 i += 1

#         if bool(re.match(SERIES_SCAN_SETTINGS, line)):
#             i += 1
#             line = lines[i]
#             param = read_params_in_line(line)

#             i += 1
#             line = lines[i]
#             vals = read_params_in_line(line)

#             series_item[SCAN_SETTINGS] = dict(zip(param, vals))

#         if bool(re.match(SERIES_RECON_SETTINGS, line)):
#             i += 1
#             line = lines[i]
#             param = read_params_in_line(line)

#             i += 1
#             line = lines[i]
#             vals = read_params_in_line(line)

#             series_item[RECON].append(dict(zip(param, vals)))


In [96]:
for k in exams:
    print('\n', k)
    for dict_ in exams[k][SERIES]:
        print()
        for i in dict_.items():
            print(i)


 ADULT HEAD 1.1 Head (adult) 1.25

('id', 'Series 2')
('type', 'Axial')
('recons', [{'Group': '1', 'DFOV': '50.0  D ', 'A/P': 'A0.0  D ', 'R/L': 'R0.0  D ', 'Filter': 'Standard', 'Type': 'Full', 'Vari': 'No', 'D3D': 'No', 'DMPR': 'No', 'Neuro': 'No', 'ASIR': 'None', 'IQEnhance': 'No', 'GSI': 'No', 'Flip': 'None', '\n': '\n'}, {'Group': '1', 'Images': '1161', 'Type': 'Full', 'Start': 'S150.0  D ', 'End': 'I1300.0  D ', 'Thick': '1.25', 'Int': '1.25', 'DFOV': '50.0  D ', 'A/P': 'A0.0  D ', 'R/L': 'R0.0  D ', 'Filter': 'Standard', 'ASIR': 'None', 'IQEnhanced': 'No', 'GSI': 'No', 'Flip': 'None\n'}, {'Group': '1', 'Images': '1161', 'Type': 'Full', 'Start': 'S150.0  D ', 'End': 'I1300.0  D ', 'Thick': '1.25', 'Int': '1.25', 'DFOV': '50.0  D ', 'A/P': 'A0.0  D ', 'R/L': 'R0.0  D ', 'Filter': 'Soft', 'ASIR': 'None', 'IQEnhanced': 'Yes', 'GSI': 'No', 'Flip': 'None\n'}, {'Group': '1', 'Images': '1161', 'Type': 'Full', 'Start': 'S150.0  D ', 'End': 'I1300.0  D ', 'Thick': '1.25', 'Int': '1.25', 

In [97]:
from pathlib import Path


def save_protocols_names(filename, exams: dict):
    lines = []
    for name in exams.keys():

        lines.append(name)

    file_name = Path(filename).stem + '_protocols_list.csv'
    output_filename = PROTOCOLS_LIST / file_name

    with open(output_filename, 'w', encoding='utf8') as output_file:
        output_file.writelines('\n'.join(lines))


# for key, val in exams.items():
#     print(key)


save_protocols_names(filename=all_protocols_path, exams=exams)

In [98]:
key = 'ADULT CHEST 5.7 Thorax_Contrast (adult) N(Free)+N(DIBH)+C(A+V) 1.25mm'
exam = exams[key]

print('exam is ', type(exam))
print('keys of exam')
for key in exam.keys():
    print(key, end=', ')
print()
print()

print('exam[SERIES] is ', type(exam[SERIES]))

print('exam[SERIES] elements:')
for ser in  exam[SERIES]:
    print(ser[ID])
    print(ser['Scan Settings'].keys())
    ser_test = ser['Scan Settings']

print()

exam is  <class 'dict'>
keys of exam
series, Exam Dose Settings, 

exam[SERIES] is  <class 'list'>
exam[SERIES] elements:
Series 2
dict_keys(['Group', 'Images', 'Speed', 'Type', 'Start', 'End', 'Thick', 'Rows', 'Int', 'HiRes', 'Shuttle', 'Tilt', 'SFOV', 'kV', 'mA', 'Message', 'Light', 'Timer', 'CTDI NV', 'CTDI', 'DLP NV', 'DLP', '\n'])
Series 9
dict_keys(['Group', 'Images', 'Speed', 'Type', 'Start', 'End', 'Thick', 'Rows', 'Int', 'HiRes', 'Shuttle', 'Tilt', 'SFOV', 'kV', 'mA', 'Message', 'Light', 'Timer', 'CTDI NV', 'CTDI', 'DLP NV', 'DLP', '\n'])
Series 16
dict_keys(['Group', 'Images', 'Speed', 'Type', 'Start', 'End', 'Thick', 'Rows', 'Int', 'HiRes', 'Shuttle', 'Tilt', 'SFOV', 'kV', 'SmartmA', 'NoiseIndex', 'MinmA', 'MaxmA', 'Message', 'Light', 'Timer', 'CTDI NV', 'CTDI', 'DLP NV', 'DLP', '\n'])



In [56]:
ser_test.keys()

dict_keys(['Group', 'Images', 'Speed', 'Type', 'Start', 'End', 'Thick', 'Rows', 'Int', 'HiRes', 'Shuttle', 'Tilt', 'SFOV', 'kV', 'SmartmA', 'NoiseIndex', 'MinmA', 'MaxmA', 'Message', 'Light', 'Timer', 'CTDI NV', 'CTDI', 'DLP NV', 'DLP', '\n'])

In [71]:
cols_output = ['Exam name', 'Series', 'Recon',
               'Slice thikness', 'DFOV', 'mA', 'kV', 'CTDI', 'DLP']
outputs = []

# add data to list
for exam_name, exam in exams.items():
    for ser in exam[SERIES]:
        for r in ser[RECON]:
            output = []
            output.append(exam_name)
            output.append(ser[ID])
            output.append(r['Group'])
            output.append(ser['Scan Settings']['Thick'])
            output.append(r['DFOV'])
            if 'mA' in ser['Scan Settings'].keys():
                output.append(ser['Scan Settings']['mA'])
            elif 'SmartmA' in ser['Scan Settings'].keys():
                output.append('SmartmA')
            else:
                output.append(',')
            output.append(ser['Scan Settings']['kV'])
            output.append(ser['Scan Settings']['CTDI'])
            output.append(ser['Scan Settings']['DLP'])


            outputs.append(output)
    outputs.append([''])

# print data

for col in cols_output:
    print(col, end=', ')

print()

for output in outputs:
    for i in output:
        if i:
            print(i, end=', ')
        else:
            print('')
    print()

Exam name, Series, Recon, Slice thikness, DFOV, mA, kV, CTDI, DLP, 
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, 1.25, 50.0  D , 340, 120, 56.1357, 8211.25, 
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, 1.25, 50.0  D , 340, 120, 56.1357, 8211.25, 
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, 1.25, 50.0  D , 340, 120, 56.1357, 8211.25, 
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, 1.25, 50.0  D , 340, 120, 56.1357, 8211.25, 


ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, 1.25, 50.0, 340, 120, 55.2667, 1175.8, 
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, 1.25, 50.0  D , 340, 120, 55.2667, 1175.8, 
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, 1.25, 50.0  D , 340, 120, 55.2667, 1175.8, 
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, 1.25, 50.0  D , 340, 120, 55.2667, 1175.8, 
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 9, 1, 1.25, 50.0  D , 340, 120, 55.2667, 1175.8, 


ADULT HEAD 1.4 Head SRS (adult) 0.625, Serie

In [105]:
cols_output = ['Exam name', 'Series', 'Recon', 'Series Description', 'Filter', 'MAR',
               'Slice thikness', 'DFOV', 'mA', 'kV', 'CTDI', 'DLP']
outputs = []

def read_protocols_names(p_path):
    with open(rt_protocols_path, 'r', encoding='utf8') as file:
        protocols = file.readlines()

    protocols = list(map(lambda x: x.strip('\n'), protocols))

    return protocols


rt_protocols_path = PROTOCOLS_LIST / \
    '2022-04-18- GE. Discovery.RT_protocols_list.csv'

rt_protocols = read_protocols_names(rt_protocols_path)


# with open(rt_protocols_path, 'r', encoding='utf8') as file:
#     rt_protocols = file.readlines()

# rt_protocols = list(map(lambda x: x.strip('\n'), rt_protocols))

# add data to list accoriding protocol list

outputs.append(cols_output)
for exam_name in rt_protocols:
    exam = exams[exam_name]
    for ser in exam[SERIES]:
        for r in ser[RECON]:
            output = []
            output.append(exam_name)
            output.append(ser[ID])
            output.append(r['Group'])
            output.append('')  # Name Description
            output.append(r['Filter'])
            output.append('')  # MAR
            output.append(ser['Scan Settings']['Thick'])
            output.append(r['DFOV'])
            if 'mA' in ser['Scan Settings'].keys():
                output.append(ser['Scan Settings']['mA'])
            elif 'SmartmA' in ser['Scan Settings'].keys():
                output.append('SmartmA')
            else:
                output.append(',')
            output.append(ser['Scan Settings']['kV'])
            output.append(ser['Scan Settings']['CTDI'])
            output.append(ser['Scan Settings']['DLP'])

            outputs.append(output)
    outputs.append([''])

# print data

# for col in cols_output:
#     print(col, end=', ')
# print()

for output in outputs:
    print(', '.join(output))

def save_protocols_table(fpath, outputs):

    lines = []
    for output in outputs:
        lines.append(', '.join(output) + '\n')

    with open(fpath, 'w', encoding='utf8') as file:
        file.writelines(lines)


rt_protocols_table_path = OUTPUT / 'RT_protocols.csv'

save_protocols_table(rt_protocols_table_path, outputs=outputs)

Exam name, Series, Recon, Series Description, Filter, MAR, Slice thikness, DFOV, mA, kV, CTDI, DLP
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, , Standard, , 1.25, 50.0  D , 340, 120, 56.1357, 8211.25
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, , Standard, , 1.25, 50.0  D , 340, 120, 56.1357, 8211.25
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, , Soft, , 1.25, 50.0  D , 340, 120, 56.1357, 8211.25
ADULT HEAD 1.1 Head (adult) 1.25, Series 2, 1, , Soft, , 1.25, 50.0  D , 340, 120, 56.1357, 8211.25

ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, , Standard, , 1.25, 50.0, 340, 120, 55.2667, 1175.8
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, , Standard, , 1.25, 50.0  D , 340, 120, 55.2667, 1175.8
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, , Soft, , 1.25, 50.0  D , 340, 120, 55.2667, 1175.8
ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C, Series 2, 1, , Soft, , 1.25, 50.0  D , 340, 120, 55.2667, 1175.8
ADULT HEAD 1.2 Head Contrast (adult) 1.2

In [93]:
outputs

[['ADULT HEAD 1.1 Head (adult) 1.25',
  'Series 2',
  '1',
  '1.25',
  '50.0  D ',
  '340',
  '120',
  '56.1357',
  '8211.25'],
 ['ADULT HEAD 1.1 Head (adult) 1.25',
  'Series 2',
  '1',
  '1.25',
  '50.0  D ',
  '340',
  '120',
  '56.1357',
  '8211.25'],
 ['ADULT HEAD 1.1 Head (adult) 1.25',
  'Series 2',
  '1',
  '1.25',
  '50.0  D ',
  '340',
  '120',
  '56.1357',
  '8211.25'],
 ['ADULT HEAD 1.1 Head (adult) 1.25',
  'Series 2',
  '1',
  '1.25',
  '50.0  D ',
  '340',
  '120',
  '56.1357',
  '8211.25'],
 [''],
 ['ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C',
  'Series 2',
  '1',
  '1.25',
  '50.0',
  '340',
  '120',
  '55.2667',
  '1175.8'],
 ['ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C',
  'Series 2',
  '1',
  '1.25',
  '50.0  D ',
  '340',
  '120',
  '55.2667',
  '1175.8'],
 ['ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C',
  'Series 2',
  '1',
  '1.25',
  '50.0  D ',
  '340',
  '120',
  '55.2667',
  '1175.8'],
 ['ADULT HEAD 1.2 Head Contrast (adult) 1.25 N+C',
  'Series 2',
